In [0]:
input_shape = (299, 299, 3)
BATCH_SIZE = 16

trainPath = os.path.sep.join(['dataset/dfdc' ,"training"])
valPath = os.path.sep.join(['dataset/dfdc' ,"validation"])
testPath = os.path.sep.join(['dataset/dfdc', "evaluation"])

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=3,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=False,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# def to prevent from bad input images
def my_gen(gen):
  i = 0
  while True:
    try:
        data, labels = next(gen)
        yield data, labels
    except:
        print(i)
        i += 1
        pass


# initialize the training generator
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="binary",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="binary",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="binary",
	target_size=(299, 299),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

labels = (trainGen.class_indices)
print(labels)

Found 149553 images belonging to 2 classes.
Found 18691 images belonging to 2 classes.
Found 18705 images belonging to 2 classes.
{'FAKE': 0, 'REAL': 1}


In [0]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
# from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# googleNet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)


hidden_dim = 256
hidden_dim1 = 128
googleNet_model = InceptionResNetV2( include_top=False, weights='imagenet', input_shape=(299, 299, 3))

# version 1
last_layer = googleNet_model.get_layer('conv_7b_ac').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim1, activation='relu', name='fc7')(x)
x = Dropout(0.2)(x)
out = Dense(1, activation='sigmoid', name='fc8')(x)
custom_googleNet_model = Model(googleNet_model.input, out)

for layer in custom_googleNet_model.layers:
	layer.trainable = False
for layer in custom_googleNet_model.layers[782:]:
	layer.trainable = True

#version 2
googleNet_model.trainable = True
custom_googleNet_model = Sequential()
custom_googleNet_model.add(googleNet_model)
custom_googleNet_model.add(GlobalAveragePooling2D())
custom_googleNet_model.add(Dense(units=1, activation='sigmoid'))


custom_googleNet_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False),
              metrics=['accuracy'])
custom_googleNet_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 8, 8, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1537      
Total params: 54,338,273
Trainable params: 54,277,729
Non-trainable params: 60,544
_________________________________________________________________
